# 桩号预测

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from config import *
context = Context('validation', gpu_start=4, dep_var=['桩号'], debug=0, mulr=1, fn_feather='tbmData/feather/removed_redundent1-full-30-no_noise-zhuanghao.feather')
databunch = MultiDeptTabularDataBunch.from_df('tmp', context.cyc_cont, context.dep_var, valid_idx=context.valid_idx_tile, bs=context.bs)

preload_pytorch()
record = False

In [ ]:
hyper_params = {
    'n_hidden': 100,
    'n_layers': 3,
    'hidden_p': 0.3,
    'input_p': 0.6,
    'weight_p': 0.5,
    'layers': [3*context.n_cont, 5, len(context.dep_var)],
    'drops': [0.2] * 3,
    'alpha': 2.,
    'beta': 1.,
    'clip': 0.5,
    'loss_func': MSELossFlat(),
#     'loss_func': l1
}

learner = get_cont_model(context, databunch, hyper_params)
learner.load('removed_redundent1-full-30-7-zhuanghao-l2');

In [ ]:
result = valid(learner, context, DatasetType.Valid)

In [ ]:
sns.distplot(result.iloc[:,0] - result.iloc[:,1], )

# Removing redundent columns

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from config import *
gpu_start = 4
# context = Context('removed_redundent1-full-30-7', gpu_start=gpu_start, dep_var=dep_var1, debug=0, mulr=7)
context = Context('removed_redundent1-full-30-no_noise', gpu_start=gpu_start, dep_var=dep_var1, debug=0, mulr=1)
valid_size = int(context.num_cycles * context.valid_ratio * context.mulr)
databunch = MultiDeptTabularDataBunch.from_df('tmp', context.cyc_cont, context.dep_var, valid_idx=context.valid_idx_tile, bs=valid_size)

record = False

In [ ]:
hyper_params = {
    'n_hidden': 100,
    'n_layers': 3,
    'hidden_p': 0.3,
    'input_p': 0.6,
    'weight_p': 0.5,
    'layers': [3*context.n_cont, 5, len(context.dep_var)],
    'drops': [0.2] * 3,
    'alpha': 2.,
    'beta': 1.,
    'clip': 0.5,
    'loss_func': MSELossFlat(),
#     'loss_func': l1
}

learner = get_cont_model(context, databunch, hyper_params)

In [ ]:
learner.load('removed_redundent1-full-30-7-2-l2');

In [ ]:
result, influence = valid(learner, context)

In [ ]:
# learner.load('removed_redundent1-full-30-7-2-l2');
learner.load('removed_redundent1-full-30-7-2.1-l2');

In [ ]:
result, influence = valid(learner, context, DatasetType.Valid)

In [ ]:
fmtr = DataFormatter(context, cycle_feathers=context.fn_cycles)
cycles = fmtr.cycles # shorten notation

In [ ]:
len(cycles)

In [ ]:
# ax = None
for i in range(5):
    axes = plots(cycles.loc[i,influence.index[:5]].iloc[:400], cols=5).flatten()
    for ax in axes: set_ax_font(ax)

In [ ]:
# learner.load('removed_redundent1-full-30-7-l2');
learner.load('removed_redundent1-full-30-7-mapd');

In [ ]:
result = valid(learner, context)

## Compare X of different noise levels

In [ ]:
# without noise
ax = None
for i in range(3):
    df = reconstruct(xb_np[i], context)
    ax = plots(df, ax=ax)

# Random forest

Feature importance for task 1
```
cols	imp
4250	刀盘给定转速显示值_29	0.092735
4255	给定频率_29	0.092533
4251	刀盘速度给定_29	0.090817
4247	刀盘转速电位器设定值_29	0.090264
4264	贯入度_29	0.018828
4275	推进泵压力_29	0.01063
```
I'm kind of concerned that the decision is largely dependent on the last few values of the target values. Let's plot these columns.

In [ ]:
fmtr = DataFormatter(context, cycle_feathers=context.fn_cycles)
cycles = fmtr.cycles # shorten notation

In [ ]:
columns = ['刀盘给定转速显示值', '给定频率', '刀盘速度给定', '刀盘转速电位器设定值', '贯入度', '推进泵压力']
plots(cycles.loc[0, columns]);